Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [56]:
import pandas as pd

In [57]:
wage_orig = pd.read_csv('wage-1-ee13a6b1-605e-44fe-83a6-85d3d7a0b3ae-2873619c-9efe-44e2-932a-f6fed7be056b.csv')
wage = wage_orig.copy()

In [58]:
wage['gender'] = wage['gender'].replace({0: 'F', 1: 'M'})

In [59]:
average_salary_by_gender = wage.groupby('gender')['wage'].mean()
average_salary_by_gender

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [61]:
id_counts = wage['person_id'].value_counts()
duplicated_id = id_counts[id_counts > 1].index
duplicated_records = wage[wage['person_id'].isin(duplicated_id)]
duplicated_records

,person_id,gender,wage
12,12,M,23547.663281
13,12,M,23547.663281
14,13,M,273483.609236
15,13,M,273483.609236
16,14,M,13434.655493
17,14,M,13434.655493
18,15,F,68341.332163
19,15,F,68341.332163
20,16,F,35448.040878
21,16,F,35448.040878


In [66]:
unique_wage_by_id = duplicated_records.groupby(['person_id', 'gender', 'wage'])['wage'].nunique()
unique_wage_by_id

person_id  gender  wage         
12         M       23547.663281     1
13         M       273483.609236    1
14         M       13434.655493     1
15         F       68341.332163     1
16         F       35448.040878     1
17         M       20178.985060     1
Name: wage, dtype: int64

In [9]:
wage_no_duplicates = wage.drop_duplicates()
wage_no_duplicates[12:18]

,person_id,gender,wage
12,12,M,23547.663281
14,13,M,273483.609236
16,14,M,13434.655493
18,15,F,68341.332163
20,16,F,35448.040878
22,17,M,20178.985060


6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [10]:
wage_info = wage_no_duplicates['wage'].describe()
wage_info

count      1000.000000
mean      43694.227404
std       55352.539343
min     -287418.645743
25%       14489.682367
50%       27309.529498
75%       52021.080258
max      755320.874132
Name: wage, dtype: float64

In [11]:
lower_bound = 0
upper_bound = wage_info['75%'] + 1.5 * (wage_info['75%'] - wage_info['25%'])  
cleaned_wage_data = wage_no_duplicates[(wage_no_duplicates['wage'] >= lower_bound) & (wage_no_duplicates['wage'] <= upper_bound)]
cleaned_wage_info = cleaned_wage_data['wage'].describe()
cleaned_wage_info

count       915.000000
mean      31998.899214
std       23850.371386
min         945.648458
25%       13735.574673
50%       25434.072946
75%       44345.417092
max      108107.141368
Name: wage, dtype: float64

7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [19]:
bonus_orig = pd.read_csv('bonus-1-c177edc7-6e8e-4e53-8f5d-098bd95cc4de-e6e1e463-001c-49ff-adc4-ef6500457b1e.csv')
bonus = bonus_orig.copy()

In [20]:
bonus[['person_id', 'bonus']] = bonus['person_id;bonus'].str.split(';', expand=True)
bonus = bonus.drop('person_id;bonus', axis=1, errors='ignore')

8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [21]:
wage = wage_no_duplicates.copy()
bonus['person_id'] = bonus['person_id'].astype('int64')

In [22]:
df = pd.merge(wage, bonus, on='person_id', how='outer')

9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [23]:
df.loc[:,'bonus'] = pd.to_numeric(df['bonus'], errors='coerce').astype('float64')
df.loc[:,'bonus'] = df['bonus'].fillna(0).map(lambda x: 0 if x == 0 else '{:.15f}'.format(x))

,person_id,gender,wage,bonus
0,0,M,46793.603811,33329.344099872563675
1,1,M,33481.575720,0
2,2,M,44523.699084,3192912.312918076291680
3,3,M,15995.576829,21968.579374844448466
4,4,F,10282.631224,0
...,...,...,...,...
997,995,M,66503.737185,3452.137071266251041
998,996,M,9972.956272,389278.969077793299221
999,997,F,104504.616392,53809.781661933659052
1000,998,M,98927.903076,0


In [24]:
df = df[df['wage'].isin(cleaned_wage_data['wage'])]

,person_id,gender,wage,bonus
0,0,M,46793.603811,33329.344099872563675
1,1,M,33481.575720,0
2,2,M,44523.699084,3192912.312918076291680
3,3,M,15995.576829,21968.579374844448466
4,4,F,10282.631224,0
...,...,...,...,...
997,995,M,66503.737185,3452.137071266251041
998,996,M,9972.956272,389278.969077793299221
999,997,F,104504.616392,53809.781661933659052
1000,998,M,98927.903076,0


In [35]:
df_copy = df.copy()
df_copy['bonus'] = df_copy['bonus'].astype('float64')
df_copy['total'] = df_copy['wage'] * 12 + df_copy['bonus']
df_copy['total'] = df_copy['total'].map(lambda x: '{:.15f}'.format(x))
df_copy['bonus'] = df_copy['bonus'].map(lambda x: '{:.15f}'.format(x))
df_copy.head()

,person_id,gender,wage,bonus,total
0,0,M,46793.603811,33329.344099872563675,594852.589829797740094
1,1,M,33481.575720,0.000000000000000,401778.908640628913417
2,2,M,44523.699084,3192912.312918076291680,3727196.701926810666919
3,3,M,15995.576829,21968.579374844448466,213915.501324415876297
4,4,F,10282.631224,0.000000000000000,123391.574686810519779


In [71]:
df_copy['total'] = df_copy['total'].astype('float64')
salary_agg = df_copy.groupby('gender')['total'].agg(['mean', 'median'])
mean_round_f = round(salary_agg.loc['F', 'mean'])
median_round_m = round(salary_agg.loc['M', 'median'])
salary_agg

,mean,median
gender,,
F,443185.114540,323983.560678
M,494837.961705,381843.546893


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [51]:
df = df_copy.copy()
df['total'] = df['total'].map(lambda x: '{:.15f}'.format(x))

In [52]:
df.to_csv('wage_bonus.csv', index=False)